In [7]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import sqrt

In [ ]:
OUTPUT_SUBDIR = "2"
FRAME_SET = "frames2"

yolo_json_dir = Path(f"outputs/yolov8/{OUTPUT_SUBDIR}")
baseline_json_dir = Path(f"outputs/simple_landmarks/{OUTPUT_SUBDIR}/json")
yolo_overlay_dir = Path(f"outputs/yolov8/{OUTPUT_SUBDIR}")
frames_dir = Path(f"frames/{FRAME_SET}")

In [ ]:
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def draw_keypoints(image, points, color, label_prefix=""):
    for i, (x, y) in enumerate(points):
        if x > 0 and y > 0:
            cv2.circle(image, (int(x), int(y)), 5, color, -1)
            cv2.putText(image, f"{label_prefix}{i+1}", (int(x)+5, int(y)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)

key_order = [
    "nose", "left_eye", "right_eye", "thumb",
    "pointer_finger", "middle_finger", "ring_finger", "pinky_finger"
]

yolo_to_baseline = {
    "nose": 0,
    "left_eye": 1,
    "right_eye": 2,
    "thumb": 4,
    "pointer_finger": 6,
    "middle_finger": 5,
    "ring_finger": 7,
    "pinky_finger": 8
}

In [ ]:
# Main comparison loop
all_frame_errors = []

for i in range(1, 7):
    frame_name = f"frame_{i:02d}"
    yolo_json_path = yolo_json_dir / f"{frame_name}_keypoints.json"
    baseline_path = baseline_json_dir / f"{frame_name}.json"
    
    if not yolo_json_path.exists():
        continue
    if not baseline_path.exists():
        continue
    
    with open(yolo_json_path, "r") as f:
        data = json.load(f)
    
    people = data.get("people", [])
    if not people:
        continue
    
    person = people[0]
    
    pose_keypoints = person.get("pose_keypoints_2d", [])
    pose_points_all = [(pose_keypoints[j], pose_keypoints[j+1]) 
                       for j in range(0, len(pose_keypoints), 3)]
    
    yolo_points = []
    for k in key_order:
        idx = yolo_to_baseline.get(k, -1)
        if idx >= 0 and idx < len(pose_points_all):
            yolo_points.append(pose_points_all[idx])
        else:
            yolo_points.append((0, 0))
    
    # Load baseline data
    with open(baseline_path, "r") as f:
        baseline_data = json.load(f)
    baseline_points = [(baseline_data[k]["coordinates"]["x"], baseline_data[k]["coordinates"]["y"]) 
                       for k in key_order]
    
    # Load image
    img_path = frames_dir / f"{frame_name}.jpg"
    if not img_path.exists():
        overlay_path = yolo_overlay_dir / f"{frame_name}_yolo.jpg"
        if overlay_path.exists():
            img_path = overlay_path
        else:
            continue
    
    img = cv2.imread(str(img_path))
    if img is None:
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    draw_keypoints(img, baseline_points, (0, 255, 0), "B")
    draw_keypoints(img, yolo_points, (255, 0, 0), "Y")
    
    errors = []
    for b, y in zip(baseline_points, yolo_points):
        if y[0] > 0 and y[1] > 0:
            dist = euclidean_distance(b, y)
            errors.append(dist)
            cv2.line(img, (int(b[0]), int(b[1])), (int(y[0]), int(y[1])), (255, 255, 0), 1)
        else:
            errors.append(np.nan)
    
    mean_error = np.mean([e for e in errors if not np.isnan(e)]) if any(not np.isnan(e) for e in errors) else np.nan
    max_error = np.nanmax(errors) if any(not np.isnan(e) for e in errors) else np.nan
    all_frame_errors.append(mean_error)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(f"{frame_name}: Mean error={mean_error:.2f}px | Max={max_error:.2f}px")
    plt.axis("off")
    plt.show()
    
    print(f"\nFrame {i:02d} Results")
    for k, dist in zip(key_order, errors):
        if not np.isnan(dist):
            print(f"  {k:15s}: {dist:7.2f}px")
        else:
            print(f"  {k:15s}: N/A (not detected)")
    if not np.isnan(mean_error):
        print(f"  Mean error: {mean_error:.2f}px | Max error: {max_error:.2f}px")

# Overall summary
if all_frame_errors:
    valid_errors = [e for e in all_frame_errors if not np.isnan(e)]
    if valid_errors:
        print(f"Average mean error across frames: {np.mean(valid_errors):.2f}px")
        print(f"Best (lowest) frame mean error: {np.min(valid_errors):.2f}px")
        print(f"Worst (highest) frame mean error: {np.max(valid_errors):.2f}px")